# Tool Calling

In this example we'll go over a basic application that selects between a few different tools, calls them, and uses an LLM to process the results.

Requirements:
- OpenAI API key, set as the env variable `OPENAI_API_KEY`
- [tomorrow.io](tomorrow.io) API Key, set as the env variable `TOMORROW_API_KEY`

# Imports

In [5]:
import inspect
import json
import os
from typing import Callable, Optional

import openai
import requests

from burr.core import State, action, when
from burr.core.application import ApplicationBuilder

# Set up + instantiate instrumentation
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument()

# Defining tools

Let's define a few tools that we want to use. They'll all have:

1. Primitive input, annotated with types
2. return types of dictionaries (free-form)

This way we can have some structure in the return type. You'll likely want a stricter return type for your cases

In [2]:
def _weather_tool(latitude: float, longitude: float) -> dict:
    """Queries the weather for a given latitude and longitude."""
    api_key = os.environ.get("TOMORROW_API_KEY")
    url = f"https://api.tomorrow.io/v4/weather/forecast?location={latitude},{longitude}&apikey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Failed to get weather data. Status code: {response.status_code}"}


def _text_wife_tool(message: str) -> dict:
    """Texts your wife with a message."""
    # Dummy implementation for the text wife tool
    # Replace this with actual SMS API logic
    return {"action": f"Texted wife: {message}"}


def _order_coffee_tool(
    size: str, coffee_preparation: str, any_modifications: Optional[str] = None
) -> dict:
    """Orders a coffee with the given size, preparation, and any modifications."""
    # Dummy implementation for the order coffee tool
    # Replace this with actual coffee shop API logic
    return {
        "action": (
            f"Ordered a {size} {coffee_preparation}" + "with {any_modifications}"
            if any_modifications
            else ""
        )
    }


def _fallback(response: str) -> dict:
    """Tells the user that the assistant can't do that -- this should be a fallback"""
    return {"response": response}


# Defining the tools (as OpenAI wants them)

We do a little cleverness to get types -- you'll likely want to use pydantic to get this right later on:

In [8]:
# You'll want to add more types here as needed
TYPE_MAP = {
    str: "string",
    int: "integer",
    float: "number",
    bool: "boolean",
}

# You can also consider using a library like pydantic to further integrate with OpenAI
OPENAI_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": fn_name,
            "description": fn.__doc__ or fn_name,
            "parameters": {
                "type": "object",
                "properties": {
                    param.name: {
                        "type": TYPE_MAP.get(param.annotation)
                        or "string",  # TODO -- add error cases
                        "description": param.name,
                    }
                    for param in inspect.signature(fn).parameters.values()
                },
                "required": [param.name for param in inspect.signature(fn).parameters.values()],
            },
        },
    }
    for fn_name, fn in {
        "query_weather": _weather_tool,
        "order_coffee": _order_coffee_tool,
        "text_wife": _text_wife_tool,
        "fallback": _fallback,
    }.items()
]

# Defining our actions

We're going to have:
1. An action that processes the user input
2. An action that selects the tool + parameters
3. An action that calls the tool
4. An action that formats the results

This way each is decoupled. The action that calls the tool will be "generic", and we'll be binding it to each tool (see next section)

In [3]:
@action(reads=[], writes=["query"])
def process_input(state: State, query: str) -> State:
    """Simple action to process input for the assistant."""
    return state.update(query=query)



@action(reads=["query"], writes=["tool_parameters", "tool"])
def select_tool(state: State) -> State:
    """Selects the tool + assigns the parameters. Uses the tool-calling API."""
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant. Use the supplied tools to assist the user, if they apply in any way. Remember to use the tools! They can do stuff you can't."
                "If you can't use only the tools provided to answer the question but know the answer, please provide the answer"
                "If you cannot use the tools provided to answer the question, use the fallback tool and provide a reason. "
                "Again, if you can't use one tool provided to answer the question, use the fallback tool and provide a reason. "
                "You must select exactly one tool no matter what, filling in every parameters with your best guess. Do not skip out on parameters!"
            ),
        },
        {"role": "user", "content": state["query"]},
    ]
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=OPENAI_TOOLS,
    )

    # Extract the tool name and parameters from OpenAI's response
    if response.choices[0].message.tool_calls is None:
        return state.update(
            tool="fallback",
            tool_parameters={
                "response": "No tool was selected, instead response was: {response.choices[0].message}."
            },
        )
    fn = response.choices[0].message.tool_calls[0].function

    return state.update(tool=fn.name, tool_parameters=json.loads(fn.arguments))


@action(reads=["tool_parameters"], writes=["raw_response"])
def call_tool(state: State, tool_function: Callable) -> State:
    """Action to call the tool. This will be bound to the tool function."""
    response = tool_function(**state["tool_parameters"])
    return state.update(raw_response=response)


@action(reads=["query", "raw_response"], writes=["final_output"])
def format_results(state: State) -> State:
    """Action to format the results in a usable way. Note we're not cascading in context for the chat history.
    This is largely due to keeping it simple, but you'll likely want to pass IDs around or maintain the chat history yourself
    """
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant. Your goal is to take the"
                    "data presented and use it to answer the original question:"
                ),
            },
            {
                "role": "user",
                "content": (
                    f"The original question was: {state['query']}."
                    f"The data is: {state['raw_response']}. Please format"
                    "the data and provide a response that responds to the original query."
                    "As always, be concise (tokens aren't free!)."
                ),
            },
        ],
    )

    return state.update(final_output=response.choices[0].message.content)

# Building the application

Now let's build the application. We will:
1. Add the right actions
2. Add the right transitions
3. Set up a tracker

In [4]:
app = (
    ApplicationBuilder()
    .with_actions(
        process_input,
        select_tool,
        format_results,
        query_weather=call_tool.bind(tool_function=_weather_tool),
        text_wife=call_tool.bind(tool_function=_text_wife_tool),
        order_coffee=call_tool.bind(tool_function=_order_coffee_tool),
        fallback=call_tool.bind(tool_function=_fallback),
    )
    .with_transitions(
        ("process_input", "select_tool"),
        ("select_tool", "query_weather", when(tool="query_weather")),
        ("select_tool", "text_wife", when(tool="text_wife")),
        ("select_tool", "order_coffee", when(tool="order_coffee")),
        ("select_tool", "fallback", when(tool="fallback")),
        (["query_weather", "text_wife", "order_coffee", "fallback"], "format_results"),
        ("format_results", "process_input"),
    )
    .with_entrypoint("process_input")
    .with_tracker(project="test_tool_calling", use_otel_tracing=True)
    .build()
)

# Running the app

And it works! 

In [9]:
app.visualize(output_file_path="./statemachine.png")
action, result, state = app.run(
    halt_after=["format_results"],
    inputs={"query": "What's the weather like in San Francisco?"},
)
print(state["final_output"])

Based on the data provided for San Francisco as of September 30, 2024, and subsequent days:

- **Temperature**: Currently around 25.5°C to 25.9°C.
- **Weather**: Clear skies, with cloud cover at 0%.
- **Humidity**: Ranges between 35% to 46%.
- **Wind**: Light winds from the north, shifting to the west, with speeds ranging from 1.63 to 3.48 km/h. Gusts can reach up to 4.17 km/h.
- **Visibility**: Excellent, at 16 km.
- **Pressure**: Around 1007.7 hPa.
- **UV Index**: Moderate to high, peaking at 6.

This indicates a clear, warm day with comfortable humidity levels and light winds.


# Visualizing

We can view in the UI. Make sure `burr` is runnng for this link to work

In [ ]:
from IPython.display import Markdown
url = f"[Link to UI](http://localhost:7241/project/demo_email_assistant/{app.uid})"
Markdown(url)